# Pandas Reshaping: Stack and Unstack
This notebook demonstrates how to handle hierarchical data (MultiIndex) using Pandas.
* **Stack**: Moves column headers into rows (making the table "taller").
* **Unstack**: Moves row indexes into columns (making the table "wider").

In [24]:
import pandas as pd

### 1. Load Data with Multi-Level Headers
We load a stock dataset that has two levels of headers:
1.  **Level 0:** Metrics ("Price", "Price to earnings ratio")
2.  **Level 1:** Company Names ("Facebook", "Google", "Microsoft")

We use `header=[0, 1]` to tell Pandas to treat the first two rows as headers.

In [25]:
df = pd.read_excel("stock_data.xlsx", header=[0, 1], index_col=0)
df

Price                  Price to earnings ratio (P/E)         \
Company    Facebook Google Microsoft                      Facebook Google   
2017-06-05      155    955        66                         37.10   32.0   
2017-06-06      150    987        69                         36.98   31.3   
2017-06-07      153    963        62                         36.78   31.7   
2017-06-08      155   1000        61                         36.11   31.2   
2017-06-09      156   1012        66                         37.07   30.0   

                      
Company    Microsoft  
2017-06-05     30.31  
2017-06-06     30.56  
2017-06-07     30.46  
2017-06-08     30.11  
2017-06-09     31.00

### 2. Stack Level 0
We use `stack(level=0)` to take the outermost column header ("Price", "P/E ratio") and move it into the rows (index).
* This groups the data by Date and Metric.

In [26]:
df_stack = df.stack(level=0)
df_stack

Company                                   Facebook  Google  Microsoft
2017-06-05 Price                            155.00   955.0      66.00
           Price to earnings ratio (P/E)     37.10    32.0      30.31
2017-06-06 Price                            150.00   987.0      69.00
           Price to earnings ratio (P/E)     36.98    31.3      30.56
2017-06-07 Price                            153.00   963.0      62.00
           Price to earnings ratio (P/E)     36.78    31.7      30.46
2017-06-08 Price                            155.00  1000.0      61.00
           Price to earnings ratio (P/E)     36.11    31.2      30.11
2017-06-09 Price                            156.00  1012.0      66.00
           Price to earnings ratio (P/E)     37.07    30.0      31.00

### 3. Unstack
We can reverse the operation using `unstack()`. This takes the innermost index level and moves it back to the columns, restoring the original wide format.

In [27]:
df_stack.unstack()

Company    Facebook                                Google  \
              Price Price to earnings ratio (P/E)   Price   
2017-06-05    155.0                         37.10   955.0   
2017-06-06    150.0                         36.98   987.0   
2017-06-07    153.0                         36.78   963.0   
2017-06-08    155.0                         36.11  1000.0   
2017-06-09    156.0                         37.07  1012.0   

Company                                  Microsoft  \
           Price to earnings ratio (P/E)     Price   
2017-06-05                          32.0      66.0   
2017-06-06                          31.3      69.0   
2017-06-07                          31.7      62.0   
2017-06-08                          31.2      61.0   
2017-06-09                          30.0      66.0   

Company                                   
           Price to earnings ratio (P/E)  
2017-06-05                         30.31  
2017-06-06                         30.56  
2017-06-07                         30.46  
2017-06-08                         30.11  
2017-06-09                         31.00

### 4. Working with 3 Levels of Headers
We now load a more complex dataset with **three** levels of hierarchy:
1.  **Level 0:** Category ("Price Ratios", "Income Statement")
2.  **Level 1:** Metric ("Price", "Net Sales", etc.)
3.  **Level 2:** Company ("FB", "Goog", "MS")

In [32]:
df2 = pd.read_excel("financial_data.xlsx", header=[0, 1, 2], index_col=0)
df2

Price Ratios                                  Income Statement       \
              Price           P/E Ratio                     Net Sales        
Qtr              FB  Goog  MS        FB  Goog     MS               FB Goog   
Q1 '16          155   955  66     37.10  32.0  30.31              2.6   20   
Q2 '16          150   987  69     36.98  31.3  30.56              3.1   22   
Q3 '16          153   963  62     36.78  31.7  30.46              4.3   24   
Q4 '16          155  1000  61     36.11  31.2  30.11              6.7   26   
Q1 '17          156  1012  66     37.07  30.0  31.00              8.1   31   

                                
              Net Profit        
Qtr        MS         FB  Goog  
Q1 '16  18.70       0.80  5.43  
Q2 '16  21.30       0.97  5.89  
Q3 '16  21.45       1.20  6.10  
Q4 '16  21.88       1.67  6.50  
Q1 '17  22.34       2.03  6.40

### 5. Default Stack (Innermost Level)
If we call `stack()` without arguments, it defaults to the **innermost level** (Level -1).
* In this case, it moves the **Company Names** (FB, Goog, MS) into the rows.

In [33]:
df2.stack()

Price Ratios           Income Statement           
                   Price P/E Ratio        Net Sales Net Profit
       Qtr                                                    
Q1 '16 FB            155     37.10             2.60       0.80
       Goog          955     32.00            20.00       5.43
       MS             66     30.31            18.70        NaN
Q2 '16 FB            150     36.98             3.10       0.97
       Goog          987     31.30            22.00       5.89
       MS             69     30.56            21.30        NaN
Q3 '16 FB            153     36.78             4.30       1.20
       Goog          963     31.70            24.00       6.10
       MS             62     30.46            21.45        NaN
Q4 '16 FB            155     36.11             6.70       1.67
       Goog         1000     31.20            26.00       6.50
       MS             61     30.11            21.88        NaN
Q1 '17 FB            156     37.07             8.10       2.03
       Goog         1012     30.00            31.00       6.40
       MS             66     31.00            22.34        NaN

### 6. Stack Level 0 (Category)
Here we stack the outermost level ("Price Ratios", "Income Statement").
* *Note:* This results in many `NaN` (Not a Number) values because "Income Statement" metrics don't exist for "Price Ratios" columns and vice-versa.

In [34]:
df2.stack(level=0)

Price               P/E Ratio              Net Sales  \
Qtr                         FB    Goog    MS        FB  Goog     MS        FB   
Q1 '16 Price Ratios      155.0   955.0  66.0     37.10  32.0  30.31       NaN   
       Income Statement    NaN     NaN   NaN       NaN   NaN    NaN       2.6   
Q2 '16 Price Ratios      150.0   987.0  69.0     36.98  31.3  30.56       NaN   
       Income Statement    NaN     NaN   NaN       NaN   NaN    NaN       3.1   
Q3 '16 Price Ratios      153.0   963.0  62.0     36.78  31.7  30.46       NaN   
       Income Statement    NaN     NaN   NaN       NaN   NaN    NaN       4.3   
Q4 '16 Price Ratios      155.0  1000.0  61.0     36.11  31.2  30.11       NaN   
       Income Statement    NaN     NaN   NaN       NaN   NaN    NaN       6.7   
Q1 '17 Price Ratios      156.0  1012.0  66.0     37.07  30.0  31.00       NaN   
       Income Statement    NaN     NaN   NaN       NaN   NaN    NaN       8.1   

                                     Net Profit        
Qtr                      Goog     MS         FB  Goog  
Q1 '16 Price Ratios       NaN    NaN        NaN   NaN  
       Income Statement  20.0  18.70       0.80  5.43  
Q2 '16 Price Ratios       NaN    NaN        NaN   NaN  
       Income Statement  22.0  21.30       0.97  5.89  
Q3 '16 Price Ratios       NaN    NaN        NaN   NaN  
       Income Statement  24.0  21.45       1.20  6.10  
Q4 '16 Price Ratios       NaN    NaN        NaN   NaN  
       Income Statement  26.0  21.88       1.67  6.50  
Q1 '17 Price Ratios       NaN    NaN        NaN   NaN  
       Income Statement  31.0  22.34       2.03  6.40

### 7. Stack Level 1 (Metrics)
Here we stack the middle level ("Price", "P/E Ratio", "Net Sales", etc.).
* This organizes the data so we can see all companies' values for a specific metric side-by-side.

In [35]:
df2.stack(level=1)

Price Ratios                Income Statement              
Qtr                         FB    Goog     MS               FB   Goog     MS
Q1 '16 Price            155.00   955.0  66.00              NaN    NaN    NaN
       P/E Ratio         37.10    32.0  30.31              NaN    NaN    NaN
       Net Sales           NaN     NaN    NaN             2.60  20.00  18.70
       Net Profit          NaN     NaN    NaN             0.80   5.43    NaN
Q2 '16 Price            150.00   987.0  69.00              NaN    NaN    NaN
       P/E Ratio         36.98    31.3  30.56              NaN    NaN    NaN
       Net Sales           NaN     NaN    NaN             3.10  22.00  21.30
       Net Profit          NaN     NaN    NaN             0.97   5.89    NaN
Q3 '16 Price            153.00   963.0  62.00              NaN    NaN    NaN
       P/E Ratio         36.78    31.7  30.46              NaN    NaN    NaN
       Net Sales           NaN     NaN    NaN             4.30  24.00  21.45
       Net Profit          NaN     NaN    NaN             1.20   6.10    NaN
Q4 '16 Price            155.00  1000.0  61.00              NaN    NaN    NaN
       P/E Ratio         36.11    31.2  30.11              NaN    NaN    NaN
       Net Sales           NaN     NaN    NaN             6.70  26.00  21.88
       Net Profit          NaN     NaN    NaN             1.67   6.50    NaN
Q1 '17 Price            156.00  1012.0  66.00              NaN    NaN    NaN
       P/E Ratio         37.07    30.0  31.00              NaN    NaN    NaN
       Net Sales           NaN     NaN    NaN             8.10  31.00  22.34
       Net Profit          NaN     NaN    NaN             2.03   6.40    NaN

### 8. Stack Level 2 (Company)
Here we explicitly stack the third level (Company Names).
* This is identical to the default `df2.stack()` behavior we saw earlier.

In [36]:
df2.stack(level=2)

Price Ratios           Income Statement           
                   Price P/E Ratio        Net Sales Net Profit
       Qtr                                                    
Q1 '16 FB            155     37.10             2.60       0.80
       Goog          955     32.00            20.00       5.43
       MS             66     30.31            18.70        NaN
Q2 '16 FB            150     36.98             3.10       0.97
       Goog          987     31.30            22.00       5.89
       MS             69     30.56            21.30        NaN
Q3 '16 FB            153     36.78             4.30       1.20
       Goog          963     31.70            24.00       6.10
       MS             62     30.46            21.45        NaN
Q4 '16 FB            155     36.11             6.70       1.67
       Goog         1000     31.20            26.00       6.50
       MS             61     30.11            21.88        NaN
Q1 '17 FB            156     37.07             8.10       2.03
       Goog         1012     30.00            31.00       6.40
       MS             66     31.00            22.34        NaN